In [1]:
from joblib import load
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pyautogui
from CRAWL import *
import pandas as pd
# 유사도 모델 로드 

Genre_list = ['雑学', 'スポーツ', '理系', '芸能', 'アニメゲーム', '文系']
# Genre 내용을 사용하여 모델 변수 6개를 만들고, 모델 불러오기
雑学= load('../Model/sim_雑学.joblib')
スポーツ = load('../Model/sim_スポーツ.joblib')
理系 = load('../Model/sim_理系.joblib')
芸能 = load('../Model/sim_芸能.joblib')
アニメゲーム = load('../Model/sim_アニメゲーム.joblib')
文系 = load('../Model/sim_文系.joblib')
Genre_model = [雑学, スポーツ, 理系, 芸能, アニメゲーム, 文系]

In [2]:
flag=0 # 데이터 추가 변수
# 화면 클릭 함수
def click_answer(most_similar_index):
    if most_similar_index==0 :
        pyautogui.click(950, 980)  
    elif most_similar_index==1:
        pyautogui.click(950, 1110) 
    elif most_similar_index==2:
        pyautogui.click(950, 1244)
    elif most_similar_index==3:
        pyautogui.click(950, 1363) 
        
def find_answer_location(imgpath):
    screenshot = np.array(pyautogui.screenshot())# 전체화면 캡처
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY) # O.png탐색
    result = cv2.matchTemplate(gray, cv2.imread(imgpath, 0), cv2.TM_CCOEFF_NORMED)# 이미지 검출
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)# 최대값 찾기
    return max_loc     # 최대값 위치 반환

# 정답 찾기
def find_answer():
    # 1초동안 실행하기
    for _ in range(5) :
        x,y = find_answer_location('../Img/XX.png')
        if 1050<=x<=1150 and 600<=y<=700:
            x,y = find_answer_location('../Img/O.png')
            if 940<=x<=1050 and 970<=y<=1094:
                return 1
            if 940<=x<=1050 and 1100<=y<=1210:
                return 2
            if 940<=x<=1050 and 1234<=y<=1341:
                return 3
            if 940<=x<=1050 and 1353<=y<=1484:
                return 4
        time.sleep(0.1)
    return 0

In [3]:
mismatch = pd.DataFrame(columns=['Q', 'Category','K1', 'K2', 'K3', 'K4', 'A'])
mismatch.to_csv('data.csv', index=False)


In [5]:
mismatch = pd.read_csv('data.csv')
# 문제 맞추기 시작
while True:
    txt_list = capture_screen()
    if len(txt_list)<5:
        print("선지가 없습니다.")
        break
    Question, Genre, Nominee = extract(txt_list)
    if Genre=="だっ" : Genre="雑学"
    if Genre==" アニメゲがゲーム" : Genre="アニメゲーム"
    if Genre=="茎能" : Genre="芸能"
    model_index= Genre_list.index(Genre)
    model = Genre_model[model_index]
    df = pd.read_csv(f"../Data/{Genre}.csv",usecols =["Q","K1","K2","K3","K4","A"])
    # 각 열을 문자열로 합쳐서 튜플 리스트 생성
    QnA = [(q + ":" + k1 +","+ k2 +","+ k3 +","+ k4, index, a) for index, q, k1, k2, k3, k4, a in zip(df.index, df['Q'], df['K1'], df['K2'], df['K3'], df['K4'], df["A"])]

    # 입력 문장을 예측
    input_text =",".join((Question, Nominee[0], Nominee[1], Nominee[2], Nominee[3]))
    predicted_label = model.predict([input_text])[0]
    
    

    print(f"유사한 문항 : {QnA[predicted_label]}\n1.{Nominee[0]}\n2.{Nominee[1]}\n3.{Nominee[2]}\n4.{Nominee[3]}")
    # 예측된 레이블에 해당하는 질문 가져오기
    predicted_question = QnA[predicted_label][2]

    if QnA[predicted_label][2] in Nominee:
        most_similar_index = Nominee.index(QnA[predicted_label][2])
    else :
        # 문장 벡터화
        vectorizer = TfidfVectorizer()
        nominee_vectors = vectorizer.fit_transform([predicted_question] + [nominee for nominee in Nominee])

        # 각 nominee와 예측된 문장 간의 유사도 계산
        similarities = cosine_similarity(nominee_vectors[0], nominee_vectors[1:])[0]

        # 가장 높은 유사도를 가지는 nominee의 인덱스 찾기
        most_similar_index = np.argmax(similarities)

    # 결과 출력 & 클릭
    print("가장 유사도가 높은 nominee의 인덱스:", most_similar_index)
    click_answer(most_similar_index)
    answer_num=find_answer()
    if answer_num : # 문제를 틀렸다면
        answer=Nominee[answer_num-1]
        print(f"총 데이터 개수 : {len(df)}\n질문 : {Question}\n선지 : {Nominee}\n정답-장르 : {answer} -------- {Genre}")
        mismatch.loc[len(df)] = [Question, Genre, Nominee[0], Nominee[1], Nominee[2], Nominee[3], answer]
        mismatch.to_csv('data.csv', index=False)

    time.sleep(1.5)

유사한 문항 : ('ハジメとバービーからなるお笑いコンビは何 ?:メイプル超合金,南海キャンディーズ,かつみ@eさゆり,フォーリンラブ', 2440, 'フォーリンラブ')
1.メイプル超純金
2.メイプル超合金
3.メイプル超黄金
4.メイプル超大金
가장 유사도가 높은 nominee의 인덱스: 0
총 데이터 개수 : 3397
질문 : 全身赤色の服を着たカズレーザーとぼっちゃり体型の安藤なつからなるお笑いコンビは何 ?
선지 : ['メイプル超純金', 'メイプル超合金', 'メイプル超黄金', 'メイプル超大金']
정답-장르 : メイプル超合金 -------- 芸能
유사한 문항 : ('代表曲に『ミュージック』『夜の距り子』がある、 山口一郎がボーカルを務めるバンドは何 ?:クサバナクション,バナナクション,アダナクション,サカナクション', 958, 'サカナクション')
1.女子十三楽坊
2.女子十四楽坊
3.女子十二楽坊
4.女子十一楽坊
가장 유사도가 높은 nominee의 인덱스: 0
총 데이터 개수 : 3397
질문 : 代表曲に『奇跡』『自由』がある、中国の伝統楽器演奏グループは何 ?
선지 : ['女子十三楽坊', '女子十四楽坊', '女子十二楽坊', '女子十一楽坊']
정답-장르 : 女子十二楽坊 -------- 芸能
유사한 문항 : ('西修の漫画『魔入りました! 入間くん』で、主人公の入間くんが通ってしとの】ち[ひだうの演2:幽霊,理様,天使,悪魔', 1980, '悪魔')
1.天使
2.理様
3.幽霊
4.悪魔
가장 유사도가 높은 nominee의 인덱스: 3
유사한 문항 : ('お県子のアイスは何の略 ?:アイスクリーム,シャーベット,かさきさ氷,ゼリー', 2437, 'アイスクリーム')
1.ゼリー
2.バリー
3.ドリー
4.デリー
가장 유사도가 높은 nominee의 인덱스: 0
유사한 문항 : ('中華人民共和国の笑都はどこ ?:東京,西京,北京,南京', 1159, '北京')
1.行灯
2.街灯
3.提条
4.電灯
가장 유사도가 높은 nominee의 인덱스: 0
총 데이터 개수 : 3333
질문 : 東京・浅草の浅草寺の雷門で、門の中央に吊り下げられているものは、のひ